# Introduction au package KERAS et au "Deep-Learning": réseaux de neurones convolutifs  (CNNs) & reconnaissance d'images

Dans ce notebook nous allons apprendre à manipuler les fonctions de base du [package Keras](https://keras.io/) permettant de mettre en oeuvre des algorithmes de réseaux de neurones, dits de "deep learning". Nous considérerons pour cela le problème le plus classique : catégorisation d'images par réseaux de neurones convolutifs (CNNs : Convolutional Neural Networks).
Nous travaillerons sur la base MNIST qui présente l'avantage (1) d'être de taille relativement restreinte, ce qui permet de réaliser l'analyse dans un temps raisonnable, et (2) d'être déjà disponible dans Keras, ce qui permet de gagner du temps sur des questions de mise en forme des données.

Le plan sera le suivant  :
1. **préparation des données**
2. **réseaux de neurones à une couche & régression (logistique) multinomiale**
3. **réseaux de neurones multicouches & modèles non-linéaires**
4. **réseaux de neurones convolutifs & apprentissage de représentation**

On commence par charger les packages et par bloquer la graine du générateur aléatoire par souci de reproductibilité. 
**Attention : keras faisant partie de tensorflow depuis la version 2 de tensorflow, les imports ne seront pas les mêmes si vous travaillez à partir de la version 2 ou d'une version antérieure.**
Pour cela, renseigner la variable **"tensorflow_version"** dans la cellule ci-dessous à la valeur **"V1"** ou **"V2"** en fonction de votre configuration.

In [ ]:
# specify tensorflow version
tensorflow_version = "V2"

In [ ]:
# import packages
import csv
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
# import tensorflow requirements - depending on version
if tensorflow_version == "V1":
    from keras import backend as K
    from keras.models import Model
    from keras.models import Sequential
    from keras.models import load_model
    from keras.layers import Dense, Activation, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras.utils import to_categorical
if tensorflow_version == "V2":
    from tensorflow.keras.models import Model
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.models import load_model
    from tensorflow.keras.layers import Dense, Activation, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D
    from tensorflow.keras.utils import to_categorical
# set random seed
np.random.seed(123)

# Partie 1. Préparation des données

Nous commençons par charger le jeu de données sauvegardé par le premier notebook.

In [ ]:
f = open('mnist-dataset.pickle', 'rb')
dataset = pickle.load(f)
f.close()

In [ ]:
X_train = dataset[0]
X_test  = dataset[1]
Y_train = dataset[2]
Y_test  = dataset[3]
y_train = dataset[4]
y_test  = dataset[5]

# Partie 2. Réseaux de neurones à une couche & régression logistique multinomiale

A la base des réseaux de neurones sont...les neurones.

Un neurone:
1. reçoit des variables d'entrée, notées ci-dessous $(x_1,...,x_n)$
2. calcule leur combinaison linéaire selon des poids associés à ses entrées, notés ici $(w_1,...,w_n)$
3. ajoute un terme de biais (ou intercept), noté ici $b$, au résultat de cette combinaison linéaire
4. applique une fonction $f$, dite d'activation, à ce score (égal à $\sum_{i=1}^n w_i x_i + b$) pour fournir une valeur de sortie.

<img src="pic/neuron.jpg" width="600">

L'apprentissage des paramètres du neurone consiste à optimiser ses poids $w_i$ et son terme de biais $b$.

Un neurone unique permet donc d'implémenter: 
* une régression linéaire en utilisant la fonction identité $f(x)=x$ comme fonction d'activation
* une régression logistique en utilisant la fonction logistique $f(x) = \frac{1}{1+\exp(-(\langle w,x \rangle+b))}$ comme fonction d'activation.

Il suffit alors de comparer la sortie du neurone à la vraie valeur de la réponse (via la fonction de perte adaptée: moindre carrés ou logistique) pour obtenir un terme d'erreur qu'on cherchera à minimiser selon les paramètres $(w_1,...,w_n,b)$, par des méthodes de descente de gradient.

Pour la classification multi-classe à $K$ classes, le problème est un peu plus compliqué car le modèle de régression multinomiale  met en jeu un vecteur de poids $w^{(k)}$ et un terme de biais $b_k$ par classe. Ces vecteurs sont liés entre eux par une transformation dite de "soft-max" pour définir la probabilité a posteriori d'être dans chacune des classes (voir par exemple [ici](https://en.wikipedia.org/wiki/Multinomial_logistic_regression) pour davantage de détails) : 
$$P(y = k \;| \; x) = \frac{ \exp({\langle w^{(k)},x \rangle + b_k}) } { \sum_{l=1}^K \exp({\langle w^{(l)},x \rangle + b_l}) }$$

Pour un problème à $K$ classes, le modèle de régression multinomiale se traduit donc en pratique par une couche de $K$ neurones, associée à une fonction d'activation "softmax".

### Création du réseau

Avant de créer le réseau, on commence par transformer les données. Dans cette première approche, on travaille bien à partir d'une représentation vectorielle des données (seuls les CNNs que nous verrons plus tard travailleront directement à partir des images).
On commence donc par transformer chaque image de taille 28x28 en un vecteur de taille 784, comme on l'avait fait en TP pour appliquer les méthodes SVMs, random-forest ou kNN.

In [ ]:
input_dim = 28*28 
X_train_vec = X_train.reshape(60000, input_dim) 
X_test_vec  = X_test.reshape(10000, input_dim)

On peut maintenant **créer** notre réseau à une couche permettant de réaliser une régression multinomiale:
* on commence par instancier un modèle de type "séquentiel" (le type de structure le plus classique, défini par une succession de couches).
* on introduit une couche de type "Dense" de taille 10 (notre nombre de classes). Une couche "dense" est une couche entièrement connectée : chaque neurone est connecté à toutes les variables d'entrées par un poids spécifique.

A noter qu'on doit toujours spécifier la taille des variables d'entrée de la première couche du réseau (ce qu'on peut faire via les arguments *input_shape* ou *input_dim*).

### <p style="color:red"> $\Rightarrow$ EXERCICE: définir le réseau en vous inspirant de l'exemple donné en cours.</p>


In [ ]:
# TODO : define "sequential model" implementing mutliclass / multinomial linear model
linreg = Sequential()


Une fois le réseau créé, on doit le **compiler**, ce qui consiste essentiellement :
1. à définir quelle stratégie d'optimisation utiliser
2. à spécifier la fonction de perte à utiliser

Nous utiliserons ici l'algorithme "standard" du gradient stochastique (**"sgd"** pour Stochastic Gradient Descent) et la fonction de perte de **"categorical_crossentropy"**, qui est le choix typique pour un problème de classification.

Notons qu'on peut également spécifier quelle métrique "suivre" lors de l'apprentissage du réseau (ici l' **"accuracy"**, i.e., le taux de bonne classification).

In [ ]:
linreg.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

On peut alors demander un **résumé** de la structure du réseau. Cela permet de vérifier que la dimension des entrée/sorties des différentes couches, et le nombre de paramètres associé, est bien conforme à ce qui attendu. Ce sera plus probant plus tard avec des réseaux plus complexes, mais on vérifie bien ici qu'on a bien le nombre de paramètres à apprendre attendu : (784 variables d'entrée + 1 terme de biais) x 10 neurones = 7850.

In [ ]:
linreg.summary()

### Apprentissage du réseau

Une fois le réseau compilé, il ne reste plus qu'à l'apprendre.
Pour cela, il suffit d'appeler la fonction **"fit"** en lui spécifiant deux paramètres : 
* **batch_size** : la taille des "batchs" à considérer pour l'algorithme SGD. Contrairement à une descente de gradient classique où on calcule le gradient sur l'ensemble du jeu de données avant de mettre à jour les poids, on les met à jour après avoir vu des sous-ensemble plus petits d'observations (des "batchs", voire à l'extrème après avoir vu une seule instance prise aléatoirement).
* **epochs** : le nombre d'itérations à faire sur le jeu de données.

Il n'y a pas de règle précise pour choisir ces paramètres, il sont à optimiser. Ici on fera 10 passages avec des batchs de taille 32.

In [ ]:
linreg.fit(X_train_vec, Y_train, batch_size=32, epochs=10, verbose=1)

Après chaque itération ("epoch"), Keras nous donne l'accuracy (tel que spécifié lors de la compilation) mesurée sur le jeu d'apprentissage. En pratique, cette valeur peut-être largement optimiste car les réseaux de neurones mettent en jeu de nombreux paramètres et sont donc susceptibles de sur-apprendre les données.

On peut néanmoins passer des options supplémentaires à la fonction "fit" pour qu'elle mesure cette performance sur un jeu de validation : 
* soit via l'argument **validation_data** de type **"validation_data=(X_val, Y_val)"**, spécifiant explicitement quelles données considérer pour la validation.
* soit via l'argument **validation_split**, comme ci-dessous, où on lui spécifie une proportion définissant la taille du jeu de validation.

In [ ]:
linreg.fit(X_train_vec, Y_train, batch_size=32, epochs=10, verbose=1, validation_split=0.2)

### Application du réseau sur le jeu de test et calcul des performance

Enfin, on peut appliquer le réseau appris sur nos données de test et calculer les performances associées.

On peut commencer par calculer globalement le taux de bonne classification : 

In [ ]:
score_linreg = linreg.evaluate(X_test_vec, Y_test, verbose=0)
print('Test-set loss:', score_linreg[0])
print('Test-set accuracy:', score_linreg[1])

On peut également calculer les prédictions elles mêmes et utiliser les outils scikit-learn vus en TP : 

In [ ]:
preds = linreg.predict_classes(X_test_vec, verbose=1) 
print("\n**** classification report ****")
print(classification_report(y_test, preds))
print("\n**** confusion matrix ****")
print(confusion_matrix(y_test, preds))

### Interprétation du réseau

Pour interpréter le réseau, on peut chercher à visualiser les poids des neurones.
On peut les extraire via la méthode **get_weights()** qu'on applique à la première (et unique) couche de notre réseau, qui est stockée dans le champ **layers** du modèle qu'on a créé. 

In [ ]:
weights = linreg.layers[0].get_weights()
weights[0].shape

In [ ]:
plt.figure(figsize =(20,8))
for i in range(n_classes):
    w = np.reshape(weights[0][:,i], (28,28))
    plt.subplot(2,5,i+1)
    plt.imshow(w)
    plt.title('weights of class %d' %i)
plt.show()

## <span style="color:red"> $\Rightarrow$ EXERCICE:   représenter l'évolution des performances estimées sur le jeu d'apprentissage et de validation au fil des "epochs". L'apprentissage semble t'il avoir convergé ? Reproduire la même analyse en considérant un plus grand nombre d'epochs (e.g., 50 ou 100).</span>

Pour cela il suffit de stocker l'objet retourné par la fonction "fit", qui est une liste contenant les critères de performance mesurés sur les jeux d'apprentissage et de validation. 

**En pratique, on re-définira le modèle afin de ré-initialiser ses paramètres (sinon ils sont issus de l'apprentissage précédent).**


On rejoue l'expérience (10 epochs) en stockant les performances mesurées lors de l'apprentissage.

In [ ]:
# re-create model to re-initialize parameters 
n_classes = 10 
linreg = Sequential() 
linreg.add(Dense(n_classes, input_dim=input_dim, activation='softmax')) 
linreg.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
# TODO : re-fit model, store "history" and plot training & validation accuracy


On rejoue l'expérience en considérant davantage d'epochs.

In [ ]:
# re-create model to re-initialize parmaters 
n_classes = 10 
linreg = Sequential() 
linreg.add(Dense(n_classes, input_dim=input_dim, activation='softmax')) 
linreg.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
# re-run analysis on 100 epochs
history_linreg = linreg.fit(X_train_vec, Y_train, batch_size=32, epochs=100, verbose=1, validation_split=0.2)

In [ ]:
# TODO : do the same thing considering a greater number of epochs


## <span style="color:red"> $\Rightarrow$ EXERCICE:   reproduire cette analyse sur un problème de classification binaire visant à reconnaitre les 0 des autres caractères. </span>

Il s'agit donc du même problème que celui considéré sur les random-forests et les SVMs. On pourra donc comparer les vecteurs de poids obtenus.

Attention, certains paramètres seront à adapter pour passer du cadre multi-classe au cadre binaire (e.g., la fonction de perte et la fonction d'activation - se référer à la documentation Keras).

In [ ]:
# TODO : re-run analysis in a binary setting. Define model, fit, get performance on test set and show weights


# Enfin, sauvegarder le modèle pour le réutiliser dans les notebooks à venir.

In [ ]:
linreg.save("model_multinomial")